1\. **Hurricanes per Year**

The number of hurricanes in 2005 was 15. The historic average is 6.3. Is this number signficantly different?
- Assume the number of hurricanes is random, i.e. follows the Poisson distribution.
- Assume as statistically significant a probability that has a Z score of 3 or larger with respect a normal distribution.

**Hint**: compute the probability that in a single year are observed 15 or more hurricances.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import math as math
from matplotlib.colors import LogNorm
from scipy import fftpack
from scipy import optimize

In [ ]:
def poisson_prob(n):
    return ((historic_avarage**n/math.factorial(n))*(math.e**-historic_avarage))

hurricanes = 15
historic_avarage = 6.3
x = np.linspace(0,30,100)
y = stats.norm.pdf(x,historic_avarage,historic_avarage)
plt.plot(x,y)
plt.axvline(hurricanes, linestyle='--', color='red',label="hurricanes")
plt.axvline(historic_avarage, linestyle='--', color='lightblue',label="avarage")
plt.fill_between(x,0,y,where=x>hurricanes,color='lightblue',label="P[n>15]")
plt.legend()
p14=0
i=hurricanes-1
while i > 0:
    p14=p14+poisson_prob(i)
    i-=1
total_prob = 1-p14
print(total_prob)

2\. **Pairwise t-test**

In an experiment, a group of 10 individuals agreed to participate in a study of blood pressure changes following exposure to halogen lighting. Resting systolic blood pressure was recorded for each individual. The participants were then exposed to 20 minutes in a room lit only by halogen lamps. A post-exposure systolic blood pressure reading was recorded for each individual. The results are presented in the following data set:

```python
pre = np.array([120, 132, 120, 110, 115, 128, 120, 112, 110, 100])
post = np.array([140, 156, 145, 130, 117, 148, 137, 119, 127, 135])
```

Determine whether the change in blood pressures within our sample was statistically significant.

**Hint:**
in this case, the Student's $t$-test should be performed to compare the two datasets.
Use the following test statistics:

$$T = \frac{\bar{x}_1 - \bar{x}_2}{\sigma \sqrt{\frac{2}{n}}}$$

and 

$$\sigma = \sqrt{\frac{\sigma_1^2 + \sigma_2^2}{2}}$$

In [ ]:
def numeratore(vector):
    somma=0
    mean_vector = np.mean(vector)
    for i in vector:
        somma += (i-mean_vector)**2
    return somma 

pre = np.array([120, 132, 120, 110, 115, 128, 120, 112, 110, 100])
post = np.array([140, 156, 145, 130, 117, 148, 137, 119, 127, 135])
sigma2_pre = np.var(pre, ddof=1)
sigma2_post = np.var(post, ddof=1)
pre_mean = np.mean(pre)
post_mean = np.mean(post)
sigma = np.sqrt((sigma2_post+sigma2_pre)/2)
T = (post_mean-pre_mean)/(sigma*np.sqrt(2/len(pre)))
print("T =", T)

xs = np.linspace(-5, +5, 100)
ts = stats.t.pdf(xs, len(pre) - 1)
plt.plot(xs, ts)
plt.fill_between(xs, ts, where=np.abs(xs) > np.abs(T), color='lightblue')
plt.axvline(T, linestyle='--', color='blue')
plt.axvline(-T, linestyle='--', color='blue')
plt.show()

3\. **FFT of a simple dataset**

Perform a periodicity analysis on the lynxs-hares population, i.e. determine what is the period of the population of these animals.

In [ ]:
def f(x, A, nu,offset,pi):
    return (A * np.cos(nu * x+pi))+offset

data = np.loadtxt("data/populations.txt")
year, hares, lynxes, carrots = data.T
plt.plot(year, hares)
popt1, pcov1 = optimize.curve_fit(f, year, hares, p0=[30000,0.6,45000,14], full_output=False)
plt.plot(year, f(year, *popt1), label='fit hares',color='r')
plt.legend()
print(*popt1)
print("the period is: ",round((2*math.pi)/popt1[1],3))
plt.show()

plt.plot(year, lynxes)
popt2, pcov2 = optimize.curve_fit(f, year, lynxes, p0=[35000,0.61,25000,1], full_output=False)
plt.plot(year, f(year, *popt2), label='fit lynxes',color='r')
print(*popt2)
print("the period is: ",round((2*math.pi)/popt2[1],3))
plt.legend()
plt.show()

4\. **FFT of an image**

Write a filter that removes the periodic noise from the `moonlanding.png` image by using a 2-dimensional FFT.

* Import the image as a 2D numpy array using `plt.imread("moonlanding.png")`. Examine the image with `plt.imshow()`, which is heavily contaminated with periodic noise.
* Check the documentation of the `scipy.fftpack` package, and find the method that performs a 2D FFT. Plot the spectrum (Fourier transform of) the image. **Hint**: use `LogNorm` to plot the colors in log scale:
```Python
from matplotlib.colors import LogNorm
plt.imshow(image, norm=LogNorm(vmin=5))
```
* Inspect the spectrum, and try to locate the regions of the power spectrum that contain the signal and those which contain the periodic noise. Use array slicing to set the noise regions to zero.
* Apply the inverse Fourier transform to plot the resulting image.

In [ ]:
image = plt.imread('data/E09/moonlanding.png')
plt.imshow(image)
plt.title('Original image')
plt.show()
fft_image = fftpack.fft2(image)
plt.imshow(np.abs(fft_image), norm=LogNorm(vmin=5))
plt.title('Fourier transform')
plt.colorbar()
plt.show()
tresh = 1500; #the noisy region are in the high frequency point
mask = fft_image.copy()
mask = np.where(np.abs(mask) > tresh, 0, mask)
final = fftpack.ifft2(mask).real
plt.imshow(final)
plt.title('Filter image')
plt.show()